In [103]:
# Import Block
import numpy as np
from rdkit import Chem
from openbabel import pybel
from openff.toolkit.topology import Molecule,Topology
from openff.toolkit.utils import RDKitToolkitWrapper
import openmm
from simtk.openmm import app
from openmm import unit
from openmm.app import PDBFile
from openff.toolkit.typing.engines.smirnoff import ForceField
from openmm.vec3 import Vec3
from scipy.optimize import minimize
import warnings
warnings.simplefilter("ignore")


In [2]:
# Wrapper and FF setup
# Use RDKit wrapper
rdktkw = RDKitToolkitWrapper()

# Loading setup parameters
forcefield = ForceField('openff-2.0.0.offxml')

In [8]:
# load pdb with one copy of pdb file
#off_mol = Molecule.from_pdb_and_smiles('7101899.pdb', "CC1=CN=C(C(=C1OC)C)C[S@@](=O)C2=NC3=C(N2)C=C(C=C3)OC")
off_mol = Molecule.from_pdb_and_smiles('1100249.pdb', "O=C1N([C@H]2C=C[C@@H]1[C@H]1C[C@@H]2C=C1)C(=O)O[C@H]1[C@@H](CC[C@H](C1)C)C(C)C")



/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [11]:
# load supercell pdb file (2x2x2) into topology
pdb_file = PDBFile('1100249_supercell.pdb')
off_top = Topology.from_openmm(pdb_file.topology, [off_mol])
# Create MD simulation inputs
system = forcefield.create_openmm_system(off_top)
integrator = openmm.VerletIntegrator(1*unit.femtoseconds)
platform = openmm.Platform.getPlatformByName('Reference')

/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /home/qualenal/anaconda3/envs/openFF/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [12]:
# create simulation
simulation = openmm.app.Simulation(pdb_file.topology, system, integrator, platform)
# set initial positions from pdbfile
positions = pdb_file.getPositions()
simulation.context.setPositions(positions)

In [13]:
# set reporters
pdb_reporter = openmm.app.PDBReporter('trajectory.pdb', 1)
state_data_reporter = openmm.app.StateDataReporter(
    "data.csv",
    1,
    step=True,
    potentialEnergy=True,
    temperature=True,
    density=True,
)
simulation.reporters.append(pdb_reporter)
simulation.reporters.append(state_data_reporter)

In [96]:
simulation.context.setPositions(positions)
simulation.saveState('initial')
orig_potential = simulation.context.getState(getEnergy=True).getPotentialEnergy()
#state_data_reporter.report(simulation, simulation.context.getState())
print('Initial Energy ' + str(orig_potential))
print('Minimizing Energy!')
simulation.minimizeEnergy()
min_state = simulation.context.getState(getEnergy=True, getPositions=True, getForces=True)
min_potential = min_state.getPotentialEnergy()
print('Final Energy = ' + str(min_potential))


Initial Energy 145251.60345995327 kJ/mol
Minimizing Energy!
Final Energy = 30661.357236135107 kJ/mol


In [15]:
simulation.step(1)


In [16]:
import mdtraj

initial = mdtraj.load_pdb('1100249_supercell.pdb')
final = mdtraj.load_pdb('trajectory.pdb')
rmsd = mdtraj.rmsd(initial,final)
print(rmsd[0])

0.014685853


In [99]:
# Example for tranformation of cartesian coordinates to fractional
pdb_file.topology.getUnitCellDimensions()
box_vectors = pdb_file.topology.getPeriodicBoxVectors()
print(box_vectors.value_in_unit(unit.nano*unit.meter))
A= np.array(box_vectors.value_in_unit(unit.nano*unit.meter))
print(A)
A_inv = np.linalg.inv(A)
x = np.array([[1,1,1],[1,1,1]])
y = np.matmul(A_inv,x.T).T
print(A_inv)
print(y)

(Vec3(x=2.1786000000000008, y=0.0, z=0.0), Vec3(x=0.0, y=2.562300000000001, z=0.0), Vec3(x=0.0, y=0.0, z=9.140400000000001))
[[2.1786 0.     0.    ]
 [0.     2.5623 0.    ]
 [0.     0.     9.1404]]
[[0.45901037 0.         0.        ]
 [0.         0.39027436 0.        ]
 [0.         0.         0.1094044 ]]
[[0.45901037 0.39027436 0.1094044 ]
 [0.45901037 0.39027436 0.1094044 ]]


In [109]:
positions = min_state.getPositions(asNumpy=True)
positions = positions.value_in_unit(unit.nano*unit.meter)
forces = min_state.getForces(asNumpy=True).value_in_unit(unit.kilojoule_per_mole/(unit.nano*unit.meter))
print(forces)
frac_positions = np.matmul(A_inv,positions.T).T
print(positions)


[[ 98.06594695  31.04075751  -0.95362774]
 [-13.1797226  -62.59494239  24.06208413]
 [-20.41267359  17.43178408   9.09313549]
 ...
 [ -1.84886972  11.57710212 -15.70444465]
 [  4.73893536  12.49130434  11.48827615]
 [ -3.68264048   5.89342671  -4.1527786 ]]
[[ 5.55067502e-01 -3.34752407e-03  6.03939390e-01]
 [ 5.56942512e-01 -5.00138818e-02  4.71991974e-01]
 [ 5.40995259e-01  1.13391779e-01  6.37535606e-01]
 ...
 [ 1.61547486e+00  2.71649660e+00  7.55426804e+00]
 [ 1.69997137e+00  2.69529778e+00  7.70906577e+00]
 [ 1.55633565e+00  2.59768484e+00  7.67186309e+00]]


In [81]:
print(box_vectors)
box_vector_array = np.array(box_vectors.value_in_unit(unit.nano*unit.meter)).flatten()
print(box_vector_array)
box_vector_array = np.delete(box_vector_array,[1,2,5]) #a_x,b_x,b_y,c_x,c_y,c_z
print(box_vector_array)
print(len(positions)/3)

(Vec3(x=2.1786000000000003, y=0.0, z=0.0), Vec3(x=0.0, y=2.5623000000000005, z=0.0), Vec3(x=0.0, y=0.0, z=9.1404)) nm
[2.1786 0.     0.     0.     2.5623 0.     0.     0.     9.1404]
[2.1786 0.     2.5623 0.     0.     9.1404]
1944.0


In [129]:
# Need block to feed parameters to minimizer, where forces provide 3*n derivatives of energy
# wrt position, +3 more derivatives of energy wrt box vectors
# This block is a work in progress

def box_energy(x,*args):
    # x is np array of 3*n(atoms) positional coordinates and 6 coordinates of 3 triclinical box vectors
    # Return the energy of the system (in kJ/mol)
    # *args will have the simulation context and n (number of particles)
    context = args[0]
    n = args[1]
    state = context.getState(getForces=True)
    # Build position array
    positions_arr = np.empty([n,3])
    for i in range(int(n)):
        positions_arr[i][0] = x[i*3]
        positions_arr[i][1] = x[i*3+1]
        positions_arr[i][2] = x[i*3+2]
    # Build periodic box vectors
    a = np.array([x[n*3],0,0])
    b = np.array([x[n*3+1],x[n*3+2],0])
    c = np.array([x[n*3+3],x[n*3+4],x[n*3+5]])
    # Set Context with positions and periodic boundary conditions
    context.setPositions(positions_arr)
    context.setPeriodicBoxVectors(a,b,c)
    # Return Energy
    return context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)

def jacobian(x,*args):
    #must return a n*3 + 6 size vector with derivative of energy with respect to each input parameter
    #for positions, return given forces
    context = args[0]
    n = args[1]
    print(x[n*3])
    forces = context.getState(getForces=True).getForces(asNumpy=True).value_in_unit(unit.kilojoule_per_mole/(unit.nano*unit.meter))
    energy = context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)

    # Positions
    positions_arr = np.empty([n,3])
    for i in range(int(n)):
        positions_arr[i][0] = x[i*3]
        positions_arr[i][1] = x[i*3+1]
        positions_arr[i][2] = x[i*3+2]
    # box vectors
    a = np.array([x[n*3],0,0])
    b = np.array([x[n*3+1],x[n*3+2],0])
    c = np.array([x[n*3+3],x[n*3+4],x[n*3+5]])
    epsilon = 1e-8
    # a_x
    temp_x = x
    temp_x[n*3] = x[n*3] + epsilon
    # TODO: rescale positions?
    new_energy = box_energy(temp_x,context,n)
    grad_a_x = (new_energy-energy)/epsilon

    return np.append(forces.flatten(),[grad_a_x,0,2.5,0,9])

# Dummy variables to test
x_test = np.append(positions.flatten(),[2.18,0,2.56,0,0,9.14])
n_test = len(positions)
print(x_test[n_test*3])
test = jacobian(x_test,simulation.context,n_test)
print(test[n_test*3])

2.18
2.18
6509453.478065552


In [106]:
result = minimize(box_energy,x_test,(simulation.context,n_test),options={'maxiter':200})


[2.18 0.   0.  ]
48717.17233852662
[2.18 0.   0.  ]
48717.1723366855
[2.18 0.   0.  ]
48717.17234036081
[2.18 0.   0.  ]
48717.17233714793
[2.18 0.   0.  ]
48717.17233916588
[2.18 0.   0.  ]
48717.17233877632
[2.18 0.   0.  ]
48717.17233916097
[2.18 0.   0.  ]
48717.172339084915
[2.18 0.   0.  ]
48717.17233712936
[2.18 0.   0.  ]
48717.1723390666
[2.18 0.   0.  ]
48717.1723395815
[2.18 0.   0.  ]
48717.17233927967
[2.18 0.   0.  ]
48717.17233482843
[2.18 0.   0.  ]
48717.172338690834
[2.18 0.   0.  ]
48717.17233930928
[2.18 0.   0.  ]
48717.17233944072
[2.18 0.   0.  ]
48717.1723385696
[2.18 0.   0.  ]
48717.17233695944
[2.18 0.   0.  ]
48717.17233892632
[2.18 0.   0.  ]
48717.172338312004
[2.18 0.   0.  ]
48717.17233820978
[2.18 0.   0.  ]
48717.17233932948
[2.18 0.   0.  ]
48717.172339427474
[2.18 0.   0.  ]
48717.17233785612
[2.18 0.   0.  ]
48717.172339703015
[2.18 0.   0.  ]
48717.17233817921
[2.18 0.   0.  ]
48717.1723378381
[2.18 0.   0.  ]
48717.172339140954
[2.18 0.   0.  ]
48

KeyboardInterrupt: 